In [10]:
import pandas as pd

In [13]:
inv_hd = pd.read_csv('inv_hd.csv')
inv_hd.head()

,store_id,invoice_id,supplier_number,invoice_number,return_invoice,status,method,cod,user_id,external_po_number,...,flags,base_total_qty,base_total_net,final_total_qty,final_misc_charge,final_misc_allowance,final_total_net,final_total_retail,num_line_items,i_audit
0,410087,170,1771394,121590806841440,False,2,2,False,76013.0,NULL,...,12289,100.0,100.00,100.0,0.0,0.0,1170.00,19900.00,1,False
1,410085,256,9179773,797761846933028,False,2,5,False,NaN,NULL,...,12289,4.0,14.69,4.0,0.0,0.0,14.84,21.96,2,False
2,410088,257,9179773,794268529783944,False,2,5,False,NaN,NULL,...,12289,158.0,718.12,158.0,0.0,0.0,725.37,9789.68,40,False
3,340028,275,4977635,974786101227895,False,2,5,False,23098.0,NULL,...,12288,1.0,119.95,12.0,0.0,0.0,119.95,155.88,1,False
4,410055,276,4977635,459915081072217,False,2,5,False,66734.0,NULL,...,12288,1.0,-295.90,198.0,0.0,0.0,3833.05,10212.60,30,False


In [14]:
inv_line = pd.read_csv('inv_line.csv')
inv_line.head()

,store_id,invoice_id,line_number,container_id,sku_number,exception,flags,base_qty,final_qty,base_um,...,base_cost,final_cost,base_allowance,final_allowance,billing_price,billing_price_mult,billing_major_dept,billing_dept_number,store_cost,store_allowance
0,340005,170,1,NaN,9881155442438,0,257.0,100.0,100.0,C,...,11.70,11.70,0.0,0.0,19.90,1.0,101.0,5.0,11.70,0.0
1,340024,256,1,NaN,7904555584739,0,36.0,2.0,2.0,E,...,3.72,3.72,0.0,0.0,5.99,1.0,1056.0,56.0,3.72,0.0
2,340024,256,2,NaN,4653609369867,256,36.0,2.0,2.0,E,...,3.70,3.70,0.0,0.0,4.99,1.0,1056.0,56.0,3.95,0.0
3,340007,257,1,NaN,6428855040557,0,36.0,1.0,1.0,E,...,3.85,3.85,0.0,0.0,3.99,1.0,719.0,57.0,3.85,0.0
4,340007,257,2,NaN,3523821409384,0,36.0,10.0,10.0,E,...,3.70,3.70,0.0,0.0,3.99,1.0,719.0,19.0,3.70,0.0


In [18]:
line_col_names = {
    'line_number': 'num_line_items',
     'base_qty': 'base_total_qty',
     'final_qty': 'final_total_qty',
     'base_cost': 'base_total_net',
     'final_cost': 'final_total_net',
     'final_allowance': 'final_misc_allowance',
     'billing_price': 'final_total_retail',
     'billing_major_dept': 'major_dept_number'
}

In [19]:
inv_line = inv_line.rename(columns=line_col_names)
inv_line.head()

,store_id,invoice_id,num_line_items,container_id,sku_number,exception,flags,base_total_qty,final_total_qty,base_um,...,base_total_net,final_total_net,base_allowance,final_misc_allowance,final_total_retail,billing_price_mult,major_dept_number,billing_dept_number,store_cost,store_allowance
0,340005,170,1,NaN,9881155442438,0,257.0,100.0,100.0,C,...,11.70,11.70,0.0,0.0,19.90,1.0,101.0,5.0,11.70,0.0
1,340024,256,1,NaN,7904555584739,0,36.0,2.0,2.0,E,...,3.72,3.72,0.0,0.0,5.99,1.0,1056.0,56.0,3.72,0.0
2,340024,256,2,NaN,4653609369867,256,36.0,2.0,2.0,E,...,3.70,3.70,0.0,0.0,4.99,1.0,1056.0,56.0,3.95,0.0
3,340007,257,1,NaN,6428855040557,0,36.0,1.0,1.0,E,...,3.85,3.85,0.0,0.0,3.99,1.0,719.0,57.0,3.85,0.0
4,340007,257,2,NaN,3523821409384,0,36.0,10.0,10.0,E,...,3.70,3.70,0.0,0.0,3.99,1.0,719.0,19.0,3.70,0.0


In [20]:
# Group the DataFrame by 'invoice_id' and calculate the maximum 'billing_dept_number'
max_dept_number = inv_line.groupby('invoice_id')['billing_dept_number'].transform('max')

# Assign the maximum 'billing_dept_number' to the 'major_dept_number' column
inv_line['major_dept_number'] = max_dept_number
inv_line.head()

,store_id,invoice_id,num_line_items,container_id,sku_number,exception,flags,base_total_qty,final_total_qty,base_um,...,base_total_net,final_total_net,base_allowance,final_misc_allowance,final_total_retail,billing_price_mult,major_dept_number,billing_dept_number,store_cost,store_allowance
0,340005,170,1,NaN,9881155442438,0,257.0,100.0,100.0,C,...,11.70,11.70,0.0,0.0,19.90,1.0,5.0,5.0,11.70,0.0
1,340024,256,1,NaN,7904555584739,0,36.0,2.0,2.0,E,...,3.72,3.72,0.0,0.0,5.99,1.0,56.0,56.0,3.72,0.0
2,340024,256,2,NaN,4653609369867,256,36.0,2.0,2.0,E,...,3.70,3.70,0.0,0.0,4.99,1.0,56.0,56.0,3.95,0.0
3,340007,257,1,NaN,6428855040557,0,36.0,1.0,1.0,E,...,3.85,3.85,0.0,0.0,3.99,1.0,57.0,57.0,3.85,0.0
4,340007,257,2,NaN,3523821409384,0,36.0,10.0,10.0,E,...,3.70,3.70,0.0,0.0,3.99,1.0,57.0,19.0,3.70,0.0


In [26]:
# Filter the DataFrame based on the condition
filtered_df = inv_line[inv_line['invoice_id'] == 257]

# Select the desired columns from the filtered DataFrame
columns_to_print = ['invoice_id', 'billing_dept_number', 'major_dept_number']

# Print the 'invoice_id' column of the filtered DataFrame
print(filtered_df[columns_to_print])

      invoice_id  billing_dept_number  major_dept_number
3            257                 57.0               57.0
4            257                 19.0               57.0
5            257                 57.0               57.0
6            257                 57.0               57.0
7            257                 57.0               57.0
...          ...                  ...                ...
5452         257                 19.0               57.0
5453         257                 19.0               57.0
5454         257                 19.0               57.0
5455         257                 19.0               57.0
5456         257                 57.0               57.0

[240 rows x 3 columns]
